In [0]:
!pip install face_recognition #installing face_recognition library
import cv2 #importing opencv library for video and image tools
import face_recognition

     |████████████████████████████████| 100.2MB 1.1MB/s 
  Stored in directory: /root/.cache/pip/wheels/d2/99/18/59c6c8f01e39810415c0e63f5bede7d83dfb0ffc039865465f
Successfully built face-recognition-models


In [0]:
from google.colab import files #uploading test video from local file system
uploaded=files.upload()

Saving test.mp4 to test.mp4


In [0]:
input_movie = cv2.VideoCapture("test.mp4") #creating a video object
length = int(input_movie.get(cv2.CAP_PROP_FRAME_COUNT)) #finding out the length of the input movie

In [0]:
codec = int(input_movie.get(cv2.CAP_PROP_FOURCC)) #4-character code of codec used to compress the frames
fps = int(input_movie.get(cv2.CAP_PROP_FPS)) #keeping a count of number of frames per second in the input video
frame_width = int(input_movie.get(cv2.CAP_PROP_FRAME_WIDTH)) #calculating frame width
frame_height = int(input_movie.get(cv2.CAP_PROP_FRAME_HEIGHT)) #calculating frame height
fourcc = cv2.VideoWriter_fourcc(*'XVID') #4 character code, usually supported by windows
output_movie = cv2.VideoWriter('output.avi',fourcc , fps, (frame_width,frame_height)) #writing an output video same as input video dimensions

In [0]:
from google.colab import files
uploaded=files.upload() #uploading training data images

Saving training.zip to training.zip


In [0]:
!unzip training

Archive:  training.zip
   creating: training/
  inflating: training/saani.jpeg     
   creating: __MACOSX/
   creating: __MACOSX/training/
  inflating: __MACOSX/training/._saani.jpeg  
  inflating: training/priyal.jpeg    
  inflating: __MACOSX/training/._priyal.jpeg  


In [0]:
import os   #preparing the training data
known_faces=[] #an array containing encoding of known faces from the training dataset
x=os.listdir("training") #opening the directory of training images
subjects=["unidentified"]
for i in x:
    image_path = "training" + "/" + i #setting the image path
    stringobj=slice(0,len(i)-5,1)
    subjects.append(i[stringobj])
    print(subjects)
    image = face_recognition.load_image_file(image_path) #loading image on the path specified above
    face_encoding = face_recognition.face_encodings(image)[0] #making face encodings 
    known_faces.append(face_encoding) #appending face  encodings to known faces array



['unidentified', 'priyal']
['unidentified', 'priyal', 'saani']


In [0]:
# Initialize variables
face_locations = []
face_encodings = []
face_names = []
frame_number = 0

In [0]:
# a function to detect and recognise faces and write an output video 
while True:
    # Grab a single frame of video
    
    ret, frame = input_movie.read()
    frame_number += 1
    
 

    # Quit when the input video file ends
    if not ret:
        break
  
    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_frame = frame[:, :, ::-1]

    # Find all the faces and face encodings in the current frame of video
    face_locations = face_recognition.face_locations(rgb_frame, model="cnn")
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)
    face_names = []
    for face_encoding in face_encodings:
        # See if the face is a match for the known face(s)
        match = face_recognition.compare_faces(known_faces, face_encoding, tolerance=0.60)
        name = None
        for x in match:
            if match[x]: #if the resulting match array matches any faces from the known faces it contains true 
               name = subjects[x+1] #name of the face from the subjects array
            else :
               name = subjects[0] #else name the face detected as unidentified 

            face_names.append(name) # append the name in the face_names array

    # Label the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
          if not name:
             continue

        # Draw a box around the face
          cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
          cv2.rectangle(frame, (left, bottom - 25), (right, bottom), (0, 0, 255), cv2.FILLED)
          font = cv2.FONT_HERSHEY_DUPLEX
          cv2.putText(frame, name, (left + 6, bottom - 6), font, 0.5, (255, 255, 255), 1)

    # Write the resulting image to the output video file
    print("Writing frame {} / {}".format(frame_number, length))
    from google.colab.patches import cv2_imshow
    cv2_imshow(frame) #showing each labelled frame
    output_movie.write(frame) #writing the output frames in output video


In [0]:
output_movie.release()
from google.colab import files   
files.download("output.avi") #downloading the output video to the local file system
input_movie.release() #releasing the input video
cv2.destroyAllWindows() #closing all the windows